In [69]:
from torch import nn
# from torch import optim
# import torch.nn.functional as F
# import numpy as np
from torchvision import datasets, transforms, models
# import torch, math
import matplotlib.pyplot as plt
import torchvision
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCEWithLogitsLoss
from torch.nn.init import xavier_uniform_
import os


In [70]:
# Create dataset
# dataset definition
from PIL import Image
import glob
import numpy as np
import os
import torch
from torchvision import transforms

class CancerDataset(Dataset):
    # load the dataset
    def __init__(self, path):
      self.X = []
      self.Y = []
      # ct = 0
      for filename in glob.glob(path+'/malignant/*'):
        im = Image.open(filename)
        self.X.append(im)
        self.Y.append(1)
        # ct += 1
        # if ct >10:
        #   break
      # print (len(self.X), len(self.Y))
      # ct = 0
      for filename in glob.glob(path+'/benign/*'):
        im=Image.open(filename)
        self.X.append(im)
        self.Y.append(0)
        # ct += 1
        # if ct >10:
        #   break

      print (len(self.X), len(self.Y))
      self.transforms = transforms.Compose([
                            transforms.Resize(112),
                            transforms.CenterCrop(112),
                            transforms.ToTensor()])

    def __len__(self):
      return len(self.X)

    def __getitem__(self, idx):
      # print (idx, self.X[idx])
      image = self.transforms(self.X[idx])
      label = self.Y[idx]
      # print (type(image), type(label))
      return (image, label)

# Prepare data
def prepare_data():
  train_path = 'drive/MyDrive/Data245proj/archive (1)/data/train'
  test_path = 'drive/MyDrive/Data245proj/archive (1)/data/test'
  trainDataset = CancerDataset(path=train_path)
  testDataset = CancerDataset(path=test_path)
  train_dl = DataLoader(trainDataset, batch_size=32, shuffle=True)
  test_dl = DataLoader(testDataset, batch_size=32, shuffle=False)
  return train_dl, test_dl


    

In [ ]:
vgg11_full_model = torch.hub.load('pytorch/vision:v0.10.0', 'vgg11', pretrained=True)
vgg11_full_model.eval()

Downloading: "https://github.com/pytorch/vision/archive/v0.10.0.zip" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/vgg11-8a719046.pth" to /root/.cache/torch/hub/checkpoints/vgg11-8a719046.pth


  0%|          | 0.00/507M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
 

In [ ]:
# helps remove the last classifier layer and only keep the backbone
# Output shape for VGGnet for 224x224 output is 4608
class VGGBackbone(nn.Module):
    def __init__(self, original_model):
        super(VGGBackbone, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-2])
        
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1)
        return x

vgg11_backbone = VGGBackbone(vgg11_full_model)
# Print model
vgg11_backbone.eval()

#Don't backprop vggnet backbone
for param in vgg11_backbone.parameters():
    param.requires_grad = False

# A, B = prepare_data()
# for i, (inputs, targets) in enumerate(A):
#   outputs = vgg11_backbone(inputs)
#   print (outputs.shape)

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        xavier_uniform_(m.weight)

# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.model = torch.nn.Sequential(Linear(n_inputs, 128),
                                          ReLU(),
                                          Linear(128, 128),
                                          ReLU(),
                                          Linear(128, 1))
        self.model.apply(weights_init)
 
    # forward propagate input
    def forward(self, X):
        X = self.model(X)
        return X
input_dim = 4608
classifier = MLP(input_dim)
classifier.eval()

MLP(
  (model): Sequential(
    (0): Linear(in_features=4608, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [ ]:

# train the model
def train_model(train_dl, model):
    # define the optimization
    criterion = BCEWithLogitsLoss()
    optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9)
    # enumerate epochs
    for epoch in range(20):
        # enumerate mini batches
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            # compute the model output
            yhat = model(inputs)
            # calculate loss
            targets = targets.reshape((-1, 1)).type(torch.float32)
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
        print("epoch: ", epoch, " loss: ", loss)
 
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = torch.sigmoid(model(inputs))
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        actual = actual.reshape((len(actual), 1))
        # round to class values
        yhat = yhat.round()
        # store
        predictions.append(yhat)
        actuals.append(actual)
    
    predictions, actuals = vstack(predictions), vstack(actuals)
    print(predictions, actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc
 
# make a class prediction for one row of data
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = torch.sigmoid(model(row))
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [72]:
# prepare the data
print ("Loading dataset...")
train_dl, test_dl = prepare_data()
print("Dataset loaded: ", len(train_dl.dataset), len(test_dl.dataset))


Loading dataset...
2637 2637
660 660
Dataset loaded:  2637 660


In [73]:
# define the network
model = torch.nn.Sequential(vgg11_backbone, classifier)
# train the model
train_model(train_dl, model)
# evaluate the model
test_acc = evaluate_model(test_dl, model)
train_acc = evaluate_model(train_dl, model)
print('Train Accuracy: %.3f' % train_acc)
print('Test Accuracy: %.3f' % test_acc)
# # make a single prediction (expect class=1)
# row = [1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1,0.03760,0.85243,-0.17755,0.59755,-0.44945,0.60536,-0.38223,0.84356,-0.38542,0.58212,-0.32192,0.56971,-0.29674,0.36946,-0.47357,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300]
# yhat = predict(row, model)
# print('Predicted: %.3f (class=%d)' % (yhat, yhat.round()))

epoch:  0  loss:  tensor(0.3057, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  1  loss:  tensor(0.1014, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  2  loss:  tensor(0.0499, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  3  loss:  tensor(0.0232, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  4  loss:  tensor(0.0204, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  5  loss:  tensor(0.0107, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  6  loss:  tensor(0.0067, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  7  loss:  tensor(0.1489, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  8  loss:  tensor(0.0143, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  9  loss:  tensor(0.0031, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  10  loss:  tensor(0.0029, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  11  loss:  tensor(0.0015, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
epoch:  12  lo